## Handle Imports

In [88]:
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats

# Upload methods

### Load Countries

In [89]:
def load_countries() -> dict:
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

### Load Resources

In [90]:
def load_resources() -> dict:
    resources = {}

    with open("resources.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Resource"]
            resources[key] = {name: float(value) if name == "Weight" else value for name, value in row.items() if name != "Resource"}

    return resources

## Upload Countries

In [91]:
countryList = load_countries()

In [92]:
resourceList = load_resources()

### Set to Pandas Dataframe

In [93]:
country_df = pd.DataFrame(countryList).transpose()

In [94]:
root_country_df = country_df

In [95]:
resource_df = pd.DataFrame(resourceList).transpose()

In [96]:
COUNTRY_NAMES = country_df.index.to_list()

In [97]:
COUNTRY_NAMES

['Atlantis', 'Dinotopoia', 'Erewhon', "King's Landing", 'The Vale']

# State Quality Function

### Check Country List

In [98]:
for key in countryList:
    print(key, '->', countryList[key])

Atlantis -> {'population': 20.0, 'food': 100.0, 'water': 300.0, 'land': 50.0, 'timber': 100.0, 'housing': 25.0, 'metallic element': 300.0, 'metallic alloy': 900.0, 'electronics': 400.0, 'potential fossil energy': 500.0, 'potential fossil usable': 250.0, 'community buildings': 200.0, 'jobs': 40.0, 'high school education': 20.0, 'college education': 12.0, 'universities': 4.0, 'nobel prizes': 3.0, 'marriages': 10.0, 'children': 10.0, 'renewable energy': 100.0, 'food waste': -5.0, 'water waste': -20.0, 'land waste': -20.0, 'timber waste': -50.0, 'housing waste': -5.0, 'metallicAlloy waste': -10.0, 'electronics waste': -14.0, 'potential fossil energy waste': -20.0, 'renewable energy waste': -1.0}
Dinotopoia -> {'population': 15.0, 'food': 100.0, 'water': 100.0, 'land': 200.0, 'timber': 300.0, 'housing': 10.0, 'metallic element': 400.0, 'metallic alloy': 100.0, 'electronics': 200.0, 'potential fossil energy': 1000.0, 'potential fossil usable': 100.0, 'community buildings': 20.0, 'jobs': 30.0

### Set Maslow Constants

In [99]:
#different resource levels
levelOneResources = {'food': 1, 'water': 1}
levelTwoResources = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1} 
levelThreeResources = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
levelFourResources = {'children': 2.5, 'renewable energy': 1}
levelFiveResources = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}

#list of levels
levList = [levelOneResources, levelTwoResources, levelThreeResources, levelFourResources, levelFiveResources]

In [100]:
print(levList)

[{'food': 1, 'water': 1}, {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1}, {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1}, {'children': 2.5, 'renewable energy': 1}, {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}]


### Level Function

In [101]:
#pandas version
def leveldf(df, country, level, resources):
    
    levelSat = True
    mult = []
    average = 0

    for key, value in levList[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = False
    
    average = stats.mean(mult)
    
    if not levelSat:
        average = average/4
        
    return average

In [102]:
num = leveldf(country_df, 'Atlantis', 1, levList)
num

200.0

## Maslow Function

In [103]:
def maslow(df, country, level):
    
    maslowList = []
    
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, levList)
        maslowList.append(levValue)
            
    return maslowList

In [104]:
maslowL = maslow(country_df, 'The Vale', 5)

In [105]:
print(maslowL)

[0.7848837209302325, 2.357364341085271, 4.944767441860465, 0.2558139534883721, 0.4127906976744186]


In [106]:
def maslowHeuristicVal(lst):
    return stats.mean(maslowL)

In [107]:
masVal = maslowHeuristicVal(maslowL)

In [108]:
print(masVal)

1.7511240310077518


## Transfer Operation

### Transform Templates

In [109]:
HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM_TM = ['FARM', {'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]

### Transform Function

In [110]:
def transform(df, country, transform_template):
    
    allowed = True
    
    #check if transform is possible
    for key in transform_template[1]:
        val = transform_template[1][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform_template[1]:
            val = transform_template[1][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform_template[2]:
            val = transform_template[2][key]
            df.loc[country, key] += val
    return allowed

### Transform Unit Test

In [111]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,100.0,300.0,50.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,250.0,20.0,100.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [112]:
transform(country_df, 'Atlantis', FARM_TM)

True

In [113]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,250.0,20.0,100.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [114]:
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)

True

In [115]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [116]:
transform(country_df, 'The Vale', FARM_TM)

False

In [117]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


# Transfer Function Definition

### Transfer Templates

In [118]:
FOOD_TR = ['FOOD', {'food': 5}]
WATER_TR = ['WATER', {'water': 5}]
TIMBER_TR = ['TIMBER', {'timber': 5}]
METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]

In [119]:
ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]

### Transfer Function

In [120]:
def transfer(df, country1, country2, transfer_template):

    allowed = True

    #check for validity
    
    if country1==country2:
        allowed = False

    for key in transfer_template[1]:
        val = transfer_template[1][key]
        if (df.loc[country1,key] - val < 0):
            allowed = False
            
    if(allowed):
        #remove resource from country 1
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country1, key] -= val

        #add resource to country 2
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country2, key] += val
            
    return allowed

## Transfer Function Test Case

In [121]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [122]:
transfer(country_df, 'Atlantis', 'The Vale', WATER_TR)

True

In [123]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,292.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,7.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [124]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

True

In [125]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

False

In [126]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


# Trade Function Definition

In [127]:
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False

## Trade Function Test Case

In [128]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [129]:
trade(country_df,'Atlantis','The Vale',FOOD_TR, WATER_TR)

False

In [130]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,280.0,2.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [131]:
trade(country_df,'Atlantis','The Vale', WATER_TR, FOOD_TR)

True

In [132]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,110.0,292.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,275.0,7.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


# Node Definition

In [133]:
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.squal
    def getExutil(self):
        return self.exutil
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
    def __lt__(self, other):
        return self.exp_util < other.exp_util

# Bounded Priority Queue Definition

In [134]:
class BoundedPriorityQueue:
    def __init__(self, bound=None):
        self.bound = bound
        self.queue = list()
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        return pq.heappop(self.queue)[-1]
    def clear(self):
        self.queue.clear()
    def len(self):
        return len(self.queue)

## Generate Succcessors

In [135]:
def generate_successors(parentNode, my_country):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for resource in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf()
        
        if transform(new_df, my_country, resource):
            state_qual = maslowHeuristicVal(maslow(new_df, my_country, 5))
            new_state = Node(state_qual, 0, prevSched, new_df)
            new_state.sched.append("TRANSFORM: " + resource[0])
            new_state.exp_util = expected_utility(new_state, my_country)
            print(new_state.exp_util)
            print(new_state)
            successors.push(new_state.exp_util, new_state)
            print(successors.len())


    for resource_1 in ALL_TEMPLATES_TRANSFER:
        for resource_2 in ALL_TEMPLATES_TRANSFER:
            for other_country in COUNTRY_NAMES:
                
                new_df = parentNode.getCountrydf()
                
                if trade(new_df, my_country, other_country, resource_1, resource_2):
                    state_qual = maslowHeuristicVal(maslow(new_df, my_country, 5))
                    new_state = Node(state_qual, 0, prevSched, new_df)
                    trade_name = "TRADE: " + my_country + " GIVES " + resource_1[0] + ", " + other_country + " GIVES " + resource_2[0]
                    new_state.sched.append(trade_name)
                    new_state.exp_util = expected_utility(new_state, my_country)
                    successors.push(new_state.exp_util, new_state)
                    
    return successors

# Reward Functions

In [136]:
def undiscounted_reward(node, my_country):
    root_node_score = maslowHeuristicVal(maslow(root_country_df, my_country, 5))
    cur_node_score = maslowHeuristicVal(maslow(node.getCountrydf(), my_country, 5))
    return cur_node_score - root_node_score

In [137]:
GAMMA = 0.05
def discounted_reward(node, my_country):
    return (GAMMA ** len(node.getSched()))*undiscounted_reward(node, my_country)

In [138]:
COST_FAILURE = -1
PROB = 0.95
def expected_utility(node, my_country):
    success = PROB * discounted_reward(node, my_country)
    fail = (1 - PROB) * COST_FAILURE
    return success + fail

# Node Test Cases 

In [139]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,110.0,292.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,275.0,7.0,94.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [140]:
state_qual = 0
exp_util = 0
sched = ['TRANSFORM', 'TRADE', 'TRANSFER', 'TRANSFORM']
test_node = Node(state_qual, exp_util, sched, country_df)

In [141]:
expected_utility(test_node, 'Atlantis')

-0.050000000000000044

In [142]:
successors = generate_successors(test_node, 'Atlantis')

-0.050000000000000044
1
-0.050000000000000044
2
-0.050000000000000044
3
-0.050000000000000044
4
-0.050000000000000044
5
-0.050000000000000044
6
-0.050000000000000044
7
-0.050000000000000044
8
-0.050000000000000044
9
-0.050000000000000044
10
-0.050000000000000044
11
-0.050000000000000044
12


In [143]:
print(successors.len())
suc1 = successors.pop()

285


In [144]:
yes = BoundedPriorityQueue()
yes.push(1, test_node)
yes.push(1, test_node)
print(yes.len())
print(yes.pop())

2


In [145]:
print(suc1.getSched())

['TRANSFORM', 'TRADE', 'TRANSFER', 'TRANSFORM', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ELECTRONICS', 'TRANSFORM: FARM', 'TRANSFORM: LOGGING', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FOSSIL_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: COMMUNITY_BUILDING', 'TRANSFORM: HIGHSCHOOL_ED', 'TRANSFORM: MARRIAGE', 'TRANSFORM: CHILDREN', 'TRADE: Atlantis GIVES FOOD, Dinotopoia GIVES WATER', 'TRADE: Atlantis GIVES FOOD, Erewhon GIVES WATER', "TRADE: Atlantis GIVES FOOD, King's Landing GIVES WATER", 'TRADE: Atlantis GIVES FOOD, The Vale GIVES WATER', 'TRADE: Atlantis GIVES FOOD, Dinotopoia GIVES TIMBER', 'TRADE: Atlantis GIVES FOOD, Erewhon GIVES TIMBER', "TRADE: Atlantis GIVES FOOD, King's Landing GIVES TIMBER", 'TRADE: Atlantis GIVES FOOD, The Vale GIVES TIMBER', 'TRADE: Atlantis GIVES FOOD, Dinotopoia GIVES METALLIC ELEMENT', 'TRADE: Atlantis GIVES FOOD, Erewhon GIVES METALLIC ELEMENT', "TRADE: Atlantis GIVES FOOD, King's Landing GIVES METALLIC ELEMENT", 'TRADE: Atlantis GI